# Tutorial #1

This notebook illustrates how [Hotcent](https://gitlab.com/mvdb/hotcent)
can be used to generate the electronic part of a DFTB parametrization
for silicon (Si), i.e.

- the eigenvalues and Hubbard values of the free atom
- the necessary two-center Hamiltonian and overlap integrals

The electronic parameters already allow to perform band structure
calculations, as shown in Tutorial #2.

Total energy calculations also require the repulsive energy.
While Hotcent also allows to build the repulsive potential from the
(confined) atomic densities, this would not work well in DFTB.
To compensate for e.g. the few-center approximations in the DFTB Hamiltonian,
it is needed to treat the repulsive potential in an empirical way,
fitting it to reproduce experimental or theoretical data.
Different tools are available for this purpose, one of which is the
[Tango](https://gitlab.com/mvdb/tango) code.

## Atomic eigenvalues

The script below shows how to set up the (all-electron) atomic DFT calculator
in Hotcent, which performs radial-symmetric DFT calculations on isolated atoms.
The eigenvalues of the $3s$ and $3p$ valence states are printed out at the end.

In [ ]:
from ase.units import Ha
from hotcent.atomic_dft import AtomicDFT

atom = AtomicDFT('Si',
                 xc='LDA',
                 configuration='[Ne] 3s2 3p2',  # the electron configuration we want to use
                 valence=['3s', '3p'],  # these will be the valence states
                 scalarrel=False,  # for Si we don't need (scalar) relativistic effects
                 perturbative_confinement=False,
                 txt='-',  # for printing to stdout
                 )
atom.run()
atom.plot_Rnl('Si_Rnl_free.png')  # plot the radial parts of the valence orbitals
atom.plot_rho('Si_rho_free.png')  # plot the valence orbital densities and total electron density

print('=======================================')
for nl in atom.valence:
    e = atom.get_eigenvalue(nl)
    print(nl, '[Ha]:', e, '[eV]:', e * Ha)

## Atomic Hubbard values

While the above eigenvalues reflect the first-order (linear) change of the
isolated atom energy as a function of the orbital occupancy, the corresponding
Hubbard values $U$ reflect the second-order (quadratic) response. By calculating
$U$ as the difference of the electron affinity (EA) and ionization energy (IE),
as done in the script below, we'll be calculating this second derivative for
the highest occupied subshell ($3p$) using second-order central differences.

Considering the LDA functional we are using, the calculated IE and EA compare
well with the [NIST](
https://webbook.nist.gov/cgi/cbook.cgi?ID=C7440213&Units=SI&Mask=20#Ion-Energetics)
reference values (IE = 8.15 eV, EA = 1.39 eV).

In [ ]:
from ase.units import Ha
from hotcent.atomic_dft import AtomicDFT
from hotcent.confinement import PowerConfinement

energies = {}
for occupation, kind in zip([2, 1, 3], ['neutral', 'cation', 'anion']):
    atom = AtomicDFT('Si',
                     xc='LDA',
                     configuration='[Ne] 3s2 3p%d' % occupation,
                     valence=['3s', '3p'],
                     scalarrel=False,
                     # Add a very weak confinement to aid anion convergence:
                     confinement=PowerConfinement(r0=40., s=4),
                     perturbative_confinement=False,
                     txt=None,
                     )
    atom.run()
    energies[kind] = atom.get_energy()

EA = energies['neutral'] - energies['anion']  # the electron affinity
IE = energies['cation'] - energies['neutral']  # the ionization energy
U = IE - EA  # the (3p) Hubbard value

for value, label in zip([EA, IE, U], ['EA', 'IE', 'U']):
    print('%2s [Ha]: %.6f    [eV]: %.6f' % (label, value, value*Ha))

For convenience you can also use the ```get_hubbard_value``` function instead:

In [ ]:
from ase.units import Ha
from hotcent.atomic_dft import AtomicDFT
from hotcent.confinement import PowerConfinement

atom = AtomicDFT('Si',
                 xc='LDA',
                 configuration='[Ne] 3s2 3p2',
                 valence=['3s', '3p'],
                 scalarrel=False,
                 # Add a very weak confinement to aid anion convergence:
                 confinement=PowerConfinement(r0=40., s=4),
                 perturbative_confinement=False,
                 txt=None,
                 )

# Like above, we use a central difference scheme
# with changes in the occupation of +/- one electron
U = atom.get_hubbard_value('3p', scheme='central', maxstep=1)
print('%2s [Ha]: %.6f    [eV]: %.6f' % ('U', value, value*Ha))

# Two-center integrals

Now we'll generate the Slater-Koster tables and save them in [SKF format](
http://www.dftb.org/fileadmin/DFTB/public/misc/slakoformat.pdf)
together with the eigenvalues, Hubbard values and subshell occupancies
in the free atom. The Slater-Koster tables are obtained by calculating
the necessary Hamiltonian and overlap integrals (here: $ss\sigma$, $sp\sigma$,
$pp\sigma$ and $pp\pi$) for a pair of Si atoms oriented along the $z$ axis.

The required wave functions and atomic densities are, in turn, obtained
by running atomic DFT calculations with added confinement.
Harmonic confinement potentials $V_\mathrm{conf} = (r/r_0)^2$ are arguably
the simplest and also the most common choice. Typical (though not necessarily
optimal) choices for the confinement radii are $r_0 = 2 R_{\mathrm{cov}}$
for the valence states and somewhat higher values for the confined density
(here $3 R_{\mathrm{cov}}$).

The integrations are performed for a range of interatomic separations,
where $0.4$ $a_0$ is a common choice for the minimal distance. The maximal
distance (here $0.4 + 600 \cdot 0.02 = 12.4$ $a_0$) should be large enough
for the integrals to approach zero.

In [ ]:
from ase.data import atomic_numbers, covalent_radii
from ase.units import Bohr
from hotcent.atomic_dft import AtomicDFT
from hotcent.confinement import PowerConfinement
from hotcent.offsite_twocenter import Offsite2cTable

# Use standard, rule-of-thumb confinement potentials
rcov = covalent_radii[atomic_numbers['Si']] / Bohr
conf = PowerConfinement(r0=3 * rcov, s=2)
wf_conf = {
    '3s': PowerConfinement(r0=2 * rcov, s=2),
    '3p': PowerConfinement(r0=2 * rcov, s=2),
}

atom = AtomicDFT('Si',
                 xc='LDA',
                 configuration='[Ne] 3s2 3p2',
                 valence=['3s', '3p'],
                 scalarrel=False,
                 confinement=conf,
                 wf_confinement=wf_conf,
                 perturbative_confinement=False,
                 txt=None,
                 )
atom.run()
atom.plot_Rnl('Si_Rnl_conf.png')
atom.plot_rho('Si_rho_conf.png')

rmin, dr, N = 0.4, 0.02, 600
off2c = Offsite2cTable(atom, atom)
off2c.run(rmin, dr, N, superposition='density', xc='LDA')

# Write the Slater-Koster tables to file (without two-body repulsion at this point).
# This file also stores the eigenvalues, Hubbardvalues, occupations, as well as the
# so-called spin-polarization error (the magnetization energy of the atom, which we
# don't need to consider here).
off2c.write(
    eigenvalues={'3s': -0.397837, '3p': -0.153163},
    hubbardvalues={'3s': 0.244242, '3p': 0.244242},
    occupations={'3s': 2, '3p': 2},
    spe=0.,
)
off2c.plot('Si-Si_offsite2c.png')
print('Done!')

Compare the radial parts of the orbitals and the orbital densities for the
free and confined atoms and have a look at the plotted Slater-Koster tables.
The resulting 'repulsionless' SKF file (```Si-Si_offsite2c.skf```) can be used
for band structure calculations.

<img src="./Si_rho_free.png" style="width: 500px;">

<img src="./Si_rho_conf.png" style="width: 500px;">

<img src="./Si-Si_offsite2c.png" style="width: 500px;">